# Coursera Capstone Final Project

### Objective:

My idea is to find the various restaurants, cafes, and other places to eat in the city of Vancouver and cluster them based on their locations and ratings. This would target a wide audience, i.e, anyone who may be looking to eat outside based on the ratings of the eateries.

### Implementation:

For this problem I will be using mainly the foursquare api and its location data. Along with that I will use the KNN clustering method for creating the clusters. 

## CODE:

In [1]:
# Import required libraries

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim
import requests
from sklearn import metrics

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print("Libraries imported")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [2]:
from bs4 import BeautifulSoup
import urllib
import urllib.request

In [3]:
# Save Foursquare API credentials

CLIENT_ID = 'OSTF3L03EQFZCVRLJY35HH3GTDPLG20PGI3UGYOZXWLASWOC'
CLIENT_SECRET = '2DRJKATHJYZPHNPNYGGNPOMLROY3EBU0D4CM4VNDC2AO2Y5Q'
VERSION = '20181020'
LIMIT = 100


### DataFrame 1

In [4]:
# address of the first dataframe

address = 'Vancouver, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


49.2608724 -123.1139529


In [5]:
search_query = 'Food'
radius = 1000
print(search_query + ' .... OK!')

Food .... OK!


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OSTF3L03EQFZCVRLJY35HH3GTDPLG20PGI3UGYOZXWLASWOC&client_secret=2DRJKATHJYZPHNPNYGGNPOMLROY3EBU0D4CM4VNDC2AO2Y5Q&ll=49.2608724,-123.1139529&v=20181020&query=Food&radius=1000&limit=100'

In [7]:
# Get json object

import json
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d66fd0da4b51b00381a16e1'},
 'response': {'venues': [{'id': '4c21219113c00f47a17b85de',
    'name': 'City Square Mall - Food Court',
    'location': {'address': '555 W. 12th Ave',
     'crossStreet': 'btwn Ash & Cambie',
     'lat': 49.26096046618133,
     'lng': -123.11635374267041,
     'labeledLatLngs': [{'label': 'display',
       'lat': 49.26096046618133,
       'lng': -123.11635374267041}],
     'distance': 174,
     'postalCode': 'V5Z 3X7',
     'cc': 'CA',
     'city': 'Vancouver',
     'state': 'BC',
     'country': 'Canada',
     'formattedAddress': ['555 W. 12th Ave (btwn Ash & Cambie)',
      'Vancouver BC V5Z 3X7',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d120951735',
      'name': 'Food Court',
      'pluralName': 'Food Courts',
      'shortName': 'Food Court',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_foodcourt_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': '

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4c21219113c00f47a17b85de,555 W. 12th Ave,CA,Vancouver,Canada,btwn Ash & Cambie,174,"[555 W. 12th Ave (btwn Ash & Cambie), Vancouve...","[{'label': 'display', 'lat': 49.26096046618133...",49.260960,-123.116354,NaN,V5Z 3X7,BC,City Square Mall - Food Court,v-1567030541
1,"[{'id': '53e0feef498e5aac066fd8a9', 'name': 'S...",False,51c738f2498e16a8c4b31057,215 West 1st Ave,CA,Vancouver,Canada,NaN,1098,"[215 West 1st Ave, Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.27070944339230...",49.270709,-123.112765,NaN,NaN,BC,Food Cart Festival,v-1567030541
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,56affd1c498e2d760992d6be,3002 Cambie St,CA,Vancouver,Canada,NaN,252,"[3002 Cambie St, Vancouver BC V5Z 2V9, Canada]","[{'label': 'display', 'lat': 49.2586971, 'lng'...",49.258697,-123.114943,NaN,V5Z 2V9,BC,Hot Delivery Chinese Food,v-1567030541
3,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,54332dc1498e491cf026e46f,Cambie,CA,NaN,Canada,West 8th,351,"[Cambie (West 8th), Canada]","[{'label': 'display', 'lat': 49.26396253446506...",49.263963,-123.114925,NaN,NaN,NaN,japadog food truck (cambie),v-1567030541
4,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4e126b1a22713f7d7bda2177,NaN,CA,Vancouver,Canada,NaN,378,"[Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.26422416445926...",49.264224,-123.114838,NaN,NaN,BC,Local Fresh Food,v-1567030541


In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df['categories'] = df.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df.columns = [column.split('.')[-1] for column in df.columns]

df.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,City Square Mall - Food Court,Food Court,555 W. 12th Ave,CA,Vancouver,Canada,btwn Ash & Cambie,174,"[555 W. 12th Ave (btwn Ash & Cambie), Vancouve...","[{'label': 'display', 'lat': 49.26096046618133...",49.260960,-123.116354,NaN,V5Z 3X7,BC,4c21219113c00f47a17b85de
1,Food Cart Festival,Street Food Gathering,215 West 1st Ave,CA,Vancouver,Canada,NaN,1098,"[215 West 1st Ave, Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.27070944339230...",49.270709,-123.112765,NaN,NaN,BC,51c738f2498e16a8c4b31057
2,Hot Delivery Chinese Food,Chinese Restaurant,3002 Cambie St,CA,Vancouver,Canada,NaN,252,"[3002 Cambie St, Vancouver BC V5Z 2V9, Canada]","[{'label': 'display', 'lat': 49.2586971, 'lng'...",49.258697,-123.114943,NaN,V5Z 2V9,BC,56affd1c498e2d760992d6be
3,japadog food truck (cambie),Food Truck,Cambie,CA,NaN,Canada,West 8th,351,"[Cambie (West 8th), Canada]","[{'label': 'display', 'lat': 49.26396253446506...",49.263963,-123.114925,NaN,NaN,NaN,54332dc1498e491cf026e46f
4,Local Fresh Food,Grocery Store,NaN,CA,Vancouver,Canada,NaN,378,"[Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.26422416445926...",49.264224,-123.114838,NaN,NaN,BC,4e126b1a22713f7d7bda2177


### DataFrame 2

In [10]:
# address of dataframe two

address2 = 'Mount Pleasant, Vancouver, CA'

geolocator2 = Nominatim(user_agent="foursquare_agent")
location2 = geolocator2.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print(latitude2, longitude2)



49.2633296 -123.0965885


In [11]:
search_query2 = 'Food'
radius2 = 1000
print(search_query2 + ' .... OK!')

Food .... OK!


In [12]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query2, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=OSTF3L03EQFZCVRLJY35HH3GTDPLG20PGI3UGYOZXWLASWOC&client_secret=2DRJKATHJYZPHNPNYGGNPOMLROY3EBU0D4CM4VNDC2AO2Y5Q&ll=49.2633296,-123.0965885&v=20181020&query=Food&radius=1000&limit=100'

In [13]:
# get json values

results2 = requests.get(url2).json()

In [14]:
# assign relevant part of JSON to venues
venues2 = results2['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues2)
dataframe2.head(100)

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,[],False,4dabc407cda1652a2baa737c,NaN,CA,Vancouver,Canada,NaN,411,"[Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.261182, 'lng':...",49.261182,-123.101192,NaN,NaN,BC,Fast Food Disco,v-1567030546
1,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,4ffb3af4e4b08c8c547c756d,Granville St.,CA,Vancouver,Canada,"Robson St,",859,"[Granville St. (Robson St,), Vancouver BC, Can...","[{'label': 'display', 'lat': 49.26592937125781...",49.265929,-123.085452,NaN,NaN,BC,Cazba Food Truck,v-1567030546
2,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",False,587d0c53dfa6ff736b9ae142,400 Industrial Ave,CA,Vancouver,Canada,NaN,667,"[400 Industrial Ave, Vancouver BC V6A 2P3, Can...","[{'label': 'display', 'lat': 49.26926070102699...",49.269261,-123.095273,NaN,V6A 2P3,BC,Movement Food,v-1567030546
3,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",False,587cae3823188e7ddb6b8fec,400 Industrial Ave,CA,Vancouver,Canada,NaN,676,"[400 Industrial Ave, Vancouver BC V6A 2P3, Can...","[{'label': 'display', 'lat': 49.2694007132226,...",49.269401,-123.096067,NaN,V6A 2P3,BC,Movement Food,v-1567030546
4,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,54d1248a498e0bdbddeb7ffb,3046 Main Street,CA,Vancouver,Canada,14th Avenue,678,"[3046 Main Street (14th Avenue), Vancouver BC,...","[{'label': 'display', 'lat': 49.25797653198242...",49.257977,-123.101044,NaN,NaN,BC,One-O-One Bulk Food & Deli,v-1567030546
5,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,5cf175b62a7ab6002cd08a70,NaN,CA,Vancouver,Canada,NaN,983,"[Vancouver BC V5T 1E1, Canada]","[{'label': 'display', 'lat': 49.26940858923988...",49.269409,-123.106404,NaN,V5T 1E1,BC,Rolling Cashew Food Truck,v-1567030546
6,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4b84b2c2f964a520784131e3,370 E Broadway,CA,Vancouver,Canada,at Kingsway,128,"[370 E Broadway (at Kingsway), Vancouver BC V5...","[{'label': 'display', 'lat': 49.26231906575304...",49.262319,-123.097452,NaN,V5T 4G5,BC,Buy-Low Foods,v-1567030546
7,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,549da4a2498e7b2806970ec0,2211 Manitoba St,CA,Vancouver,Canada,NaN,800,"[2211 Manitoba St, Vancouver BC V5Y 3A3, Canada]","[{'label': 'display', 'lat': 49.26544906308243...",49.265449,-123.107113,NaN,V5Y 3A3,BC,Invitro Food Labs and Eatery,v-1567030546
8,[],False,4ad246eff964a5209fe020e3,701 kingsway,CA,Vancouver,Canada,Fraser,898,"[701 kingsway (Fraser), Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.25680244089009...",49.256802,-123.089316,NaN,NaN,BC,alenka european food,v-1567030546
9,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,4fce5eade4b0155a2ec35251,NaN,CA,Vancouver,Canada,NaN,917,"[Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.26622766409765...",49.266228,-123.084767,NaN,NaN,BC,Triple O's Food Truck,v-1567030546


In [15]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
df2 = dataframe2.loc[:, filtered_columns2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list2 = row['categories']
    except:
        categories_list2 = row['venue.categories']
        
    if len(categories_list2) == 0:
        return None
    else:
        return categories_list2[0]['name']

# filter the category for each row
df2['categories'] = df2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df2.columns = [column.split('.')[-1] for column in df2.columns]

df2.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Fast Food Disco,None,NaN,CA,Vancouver,Canada,NaN,411,"[Vancouver BC, Canada]","[{'label': 'display', 'lat': 49.261182, 'lng':...",49.261182,-123.101192,NaN,NaN,BC,4dabc407cda1652a2baa737c
1,Cazba Food Truck,Food Truck,Granville St.,CA,Vancouver,Canada,"Robson St,",859,"[Granville St. (Robson St,), Vancouver BC, Can...","[{'label': 'display', 'lat': 49.26592937125781...",49.265929,-123.085452,NaN,NaN,BC,4ffb3af4e4b08c8c547c756d
2,Movement Food,Food & Drink Shop,400 Industrial Ave,CA,Vancouver,Canada,NaN,667,"[400 Industrial Ave, Vancouver BC V6A 2P3, Can...","[{'label': 'display', 'lat': 49.26926070102699...",49.269261,-123.095273,NaN,V6A 2P3,BC,587d0c53dfa6ff736b9ae142
3,Movement Food,Health Food Store,400 Industrial Ave,CA,Vancouver,Canada,NaN,676,"[400 Industrial Ave, Vancouver BC V6A 2P3, Can...","[{'label': 'display', 'lat': 49.2694007132226,...",49.269401,-123.096067,NaN,V6A 2P3,BC,587cae3823188e7ddb6b8fec
4,One-O-One Bulk Food & Deli,Grocery Store,3046 Main Street,CA,Vancouver,Canada,14th Avenue,678,"[3046 Main Street (14th Avenue), Vancouver BC,...","[{'label': 'display', 'lat': 49.25797653198242...",49.257977,-123.101044,NaN,NaN,BC,54d1248a498e0bdbddeb7ffb


### DataFrame 3

In [16]:
# address of dataframe three

address3 = 'Cambie, Vancouver, CA'

geolocator3 = Nominatim(user_agent="foursquare_agent")
location3 = geolocator3.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print(latitude3, longitude3)

49.2831072 -123.1091298


In [17]:
search_query3 = 'Restaurant ratings'
radius = 1000
print(search_query3 + ' .... OK!')

Restaurant ratings .... OK!


In [18]:
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude3, longitude3, VERSION, search_query3, radius, LIMIT)
url3

'https://api.foursquare.com/v2/venues/search?client_id=OSTF3L03EQFZCVRLJY35HH3GTDPLG20PGI3UGYOZXWLASWOC&client_secret=2DRJKATHJYZPHNPNYGGNPOMLROY3EBU0D4CM4VNDC2AO2Y5Q&ll=49.2831072,-123.1091298&v=20181020&query=Restaurant ratings&radius=1000&limit=100'

In [19]:
# get json values

results3 = requests.get(url3).json()
results3


{'meta': {'code': 200, 'requestId': '5d66fd17e97dfb0039de567b'},
 'response': {'venues': [{'id': '4aa7fe08f964a520914e20e3',
    'name': 'Nuba',
    'location': {'address': '207 W Hastings St',
     'crossStreet': 'at Cambie St',
     'lat': 49.28281170120566,
     'lng': -123.10958233142155,
     'labeledLatLngs': [{'label': 'display',
       'lat': 49.28281170120566,
       'lng': -123.10958233142155}],
     'distance': 46,
     'postalCode': 'V6B 2N4',
     'cc': 'CA',
     'city': 'Vancouver',
     'state': 'BC',
     'country': 'Canada',
     'formattedAddress': ['207 W Hastings St (at Cambie St)',
      'Vancouver BC V6B 2N4',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d115941735',
      'name': 'Middle Eastern Restaurant',
      'pluralName': 'Middle Eastern Restaurants',
      'shortName': 'Middle Eastern',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/middleeastern_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': '

In [20]:
# assign relevant part of JSON to venues
venues3 = results3['response']['venues']

# tranform venues into a dataframe
dataframe3 = json_normalize(venues3)
dataframe3.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",False,4aa7fe08f964a520914e20e3,207 W Hastings St,CA,Vancouver,Canada,at Cambie St,46,"[207 W Hastings St (at Cambie St), Vancouver B...","[{'label': 'display', 'lat': 49.28281170120566...",49.282812,-123.109582,NaN,V6B 2N4,BC,Nuba,v-1567030552,NaN
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4abe7fb7f964a520078e20e3,148 E Pender St,CA,Vancouver,Canada,Main St,648,"[148 E Pender St (Main St), Vancouver BC V6A 1...","[{'label': 'display', 'lat': 49.28042046714889...",49.280420,-123.101205,NaN,V6A 1T3,BC,New Town Bakery & Restaurant,v-1567030552,NaN
2,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",False,4aa82b93f964a520c84f20e3,433 West Pender,CA,Vancouver,Canada,Homer,245,"[433 West Pender (Homer), Vancouver BC V6B 1V2...","[{'label': 'display', 'lat': 49.28316092121001...",49.283161,-123.112503,NaN,V6B 1V2,BC,Yagger's Downtown Restaurant & Sports Bar,v-1567030552,41496929
3,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,4aa837e6f964a5202c5020e3,332 Water Street,CA,Vancouver,Canada,Cambie Street,143,"[332 Water Street (Cambie Street), Vancouver B...","[{'label': 'display', 'lat': 49.2843757721172,...",49.284376,-123.109509,NaN,V6B 1B6,BC,McLean's Restaurant,v-1567030552,NaN
4,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4aa7f561f964a520384e20e3,200 Granville Street,CA,Vancouver,Canada,West Cordova Street,453,"[200 Granville Street (West Cordova Street), V...","[{'label': 'display', 'lat': 49.28671261, 'lng...",49.286713,-123.112044,NaN,V6C 1S4,BC,Miku Restaurant,v-1567030552,NaN


In [21]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns3 = ['name', 'categories'] + [col for col in dataframe3.columns if col.startswith('location.')] + ['id']
df3 = dataframe3.loc[:, filtered_columns3]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue3.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
df3['categories'] = df3.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df3.columns = [column.split('.')[-1] for column in df3.columns]

df3.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Nuba,Middle Eastern Restaurant,207 W Hastings St,CA,Vancouver,Canada,at Cambie St,46,"[207 W Hastings St (at Cambie St), Vancouver B...","[{'label': 'display', 'lat': 49.28281170120566...",49.282812,-123.109582,NaN,V6B 2N4,BC,4aa7fe08f964a520914e20e3
1,New Town Bakery & Restaurant,Chinese Restaurant,148 E Pender St,CA,Vancouver,Canada,Main St,648,"[148 E Pender St (Main St), Vancouver BC V6A 1...","[{'label': 'display', 'lat': 49.28042046714889...",49.280420,-123.101205,NaN,V6A 1T3,BC,4abe7fb7f964a520078e20e3
2,Yagger's Downtown Restaurant & Sports Bar,Sports Bar,433 West Pender,CA,Vancouver,Canada,Homer,245,"[433 West Pender (Homer), Vancouver BC V6B 1V2...","[{'label': 'display', 'lat': 49.28316092121001...",49.283161,-123.112503,NaN,V6B 1V2,BC,4aa82b93f964a520c84f20e3
3,McLean's Restaurant,Breakfast Spot,332 Water Street,CA,Vancouver,Canada,Cambie Street,143,"[332 Water Street (Cambie Street), Vancouver B...","[{'label': 'display', 'lat': 49.2843757721172,...",49.284376,-123.109509,NaN,V6B 1B6,BC,4aa837e6f964a5202c5020e3
4,Miku Restaurant,Japanese Restaurant,200 Granville Street,CA,Vancouver,Canada,West Cordova Street,453,"[200 Granville Street (West Cordova Street), V...","[{'label': 'display', 'lat': 49.28671261, 'lng...",49.286713,-123.112044,NaN,V6C 1S4,BC,4aa7f561f964a520384e20e3


### Join 3 databases

In [22]:
df_new = pd.concat([df, df2, df3], sort=True)
df_new.head()

,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state
0,555 W. 12th Ave,Food Court,CA,Vancouver,Canada,btwn Ash & Cambie,174,"[555 W. 12th Ave (btwn Ash & Cambie), Vancouve...",4c21219113c00f47a17b85de,"[{'label': 'display', 'lat': 49.26096046618133...",49.260960,-123.116354,City Square Mall - Food Court,NaN,V5Z 3X7,BC
1,215 West 1st Ave,Street Food Gathering,CA,Vancouver,Canada,NaN,1098,"[215 West 1st Ave, Vancouver BC, Canada]",51c738f2498e16a8c4b31057,"[{'label': 'display', 'lat': 49.27070944339230...",49.270709,-123.112765,Food Cart Festival,NaN,NaN,BC
2,3002 Cambie St,Chinese Restaurant,CA,Vancouver,Canada,NaN,252,"[3002 Cambie St, Vancouver BC V5Z 2V9, Canada]",56affd1c498e2d760992d6be,"[{'label': 'display', 'lat': 49.2586971, 'lng'...",49.258697,-123.114943,Hot Delivery Chinese Food,NaN,V5Z 2V9,BC
3,Cambie,Food Truck,CA,NaN,Canada,West 8th,351,"[Cambie (West 8th), Canada]",54332dc1498e491cf026e46f,"[{'label': 'display', 'lat': 49.26396253446506...",49.263963,-123.114925,japadog food truck (cambie),NaN,NaN,NaN
4,NaN,Grocery Store,CA,Vancouver,Canada,NaN,378,"[Vancouver BC, Canada]",4e126b1a22713f7d7bda2177,"[{'label': 'display', 'lat': 49.26422416445926...",49.264224,-123.114838,Local Fresh Food,NaN,NaN,BC


In [23]:
# map with all venues

venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Vancouver',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df_new.lat, df_new.lng, df_new.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [24]:
# df with only lat and long values

df_van = df_new.drop(['name', 'categories', 'address', 'neighborhood', 'cc', 'country', 'crossStreet', 'formattedAddress', 'labeledLatLngs', 'state', 'city', 'postalCode'], axis=1)
df_vanc = df_van.drop(['distance', 'id'], axis=1)
df_vanc.head()

,lat,lng
0,49.260960,-123.116354
1,49.270709,-123.112765
2,49.258697,-123.114943
3,49.263963,-123.114925
4,49.264224,-123.114838


In [25]:
# set number of clusters
kclusters = 7


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_vanc)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([3, 2, 3, 3, 3, 3, 3, 3, 3, 2], dtype=int32)

In [26]:
df_vanc.insert(0, 'Cluster Labels', kmeans.labels_)
df_vanc.head()

,Cluster Labels,lat,lng
0,3,49.260960,-123.116354
1,2,49.270709,-123.112765
2,3,49.258697,-123.114943
3,3,49.263963,-123.114925
4,3,49.264224,-123.114838


In [27]:
# Map with clusters

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude3, longitude3], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_vanc['lat'], df_vanc['lng'], df_new['name'], df_vanc['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [28]:
# setup to get tips of each venue

df_blah = df_new.drop(['distance', 'categories', 'address', 'neighborhood', 'cc', 'country', 'crossStreet', 'formattedAddress', 'labeledLatLngs', 'state', 'city', 'postalCode', 'lat', 'lng'], axis=1)
venue_ratings = []
SORT = 'popular'
OFFSET = 1
list_id = df_new['id'].values
count = 0
LIMIT = 1

In [29]:
# list of tips for each venue(venue_ratings)
k = 0
for value in df_blah['name']:
    VENUE_ID = list_id[k]
    tipsurl = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&sort={}&limit={}&offset={}'.format(VENUE_ID, CLIENT_ID, CLIENT_SECRET, VERSION, SORT, 1, OFFSET)
    results = requests.get(tipsurl).json()
    venue_ratings.append(results)
    k=k+1
    count = 1


In [30]:
venue_ratings

[{'meta': {'code': 200, 'requestId': '5d66fd2d66fc65002c2c4813'},
  'response': {'tips': {'count': 1,
    'items': [{'id': '4f85de4ee4b0a3bc1668c94e',
      'createdAt': 1334173262,
      'text': 'Go to Vana Cuisine and order number 10 Lemon Grass chicken! Awesome food',
      'type': 'user',
      'canonicalUrl': 'https://foursquare.com/item/4f85de4ee4b0a3bc1668c94e',
      'lang': 'en',
      'likes': {'count': 1,
       'groups': [{'type': 'others',
         'count': 1,
         'items': [{'id': '42985',
           'firstName': 'Mark',
           'lastName': 'Dee',
           'gender': 'male',
           'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
            'suffix': '/QQ3L12HPJJFAOFHJ.jpg'}}]}],
       'summary': '1 like'},
      'logView': True,
      'agreeCount': 2,
      'disagreeCount': 0,
      'todo': {'count': 0},
      'user': {'id': '331583',
       'firstName': 'Ade',
       'lastName': 'Win',
       'gender': 'male',
       'photo': {'prefix': 'https://fa

In [31]:
# keep only wanted values
list1 = []
num = 0
for i in venue_ratings:
    ratings_venue = venue_ratings[num]['response']['tips']['items']
    list1.append(ratings_venue)
    num = num + 1

#blah = venue_ratings[0]['response']['tips']['items']
#df = json_normalize(blah)
#df

In [32]:
list1

[[{'id': '4f85de4ee4b0a3bc1668c94e',
   'createdAt': 1334173262,
   'text': 'Go to Vana Cuisine and order number 10 Lemon Grass chicken! Awesome food',
   'type': 'user',
   'canonicalUrl': 'https://foursquare.com/item/4f85de4ee4b0a3bc1668c94e',
   'lang': 'en',
   'likes': {'count': 1,
    'groups': [{'type': 'others',
      'count': 1,
      'items': [{'id': '42985',
        'firstName': 'Mark',
        'lastName': 'Dee',
        'gender': 'male',
        'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
         'suffix': '/QQ3L12HPJJFAOFHJ.jpg'}}]}],
    'summary': '1 like'},
   'logView': True,
   'agreeCount': 2,
   'disagreeCount': 0,
   'todo': {'count': 0},
   'user': {'id': '331583',
    'firstName': 'Ade',
    'lastName': 'Win',
    'gender': 'male',
    'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
     'suffix': '/L32ITWGHTDBJ2PVK.jpg'}}}],
 [{'id': '53e7e89411d22533ef8a64c9',
   'createdAt': 1407707284,
   'text': 'Come with friends. Divide and conquer. 

In [33]:
# convert to dataframe
n = 0
c = 0
for i in list1:
    df_r = json_normalize(list1[n])
    if c == 0:
        dfr = df_r
    else:
        dfr = dfr.append(df_r, sort=True)
    c = 1
    n = n + 1
    
dfr

,agreeCount,authorInteractionType,canonicalUrl,createdAt,disagreeCount,id,lang,lastUpvoteTimestamp,lastVoteText,likes.count,likes.groups,likes.summary,logView,photo.createdAt,photo.height,photo.id,photo.prefix,photo.source.name,photo.source.url,photo.suffix,photo.visibility,photo.width,photourl,text,todo.count,type,url,user.firstName,user.gender,user.id,user.lastName,user.photo.default,user.photo.prefix,user.photo.suffix,user.type
0,2,NaN,https://foursquare.com/item/4f85de4ee4b0a3bc16...,1334173262,0,4f85de4ee4b0a3bc1668c94e,en,NaN,NaN,1,"[{'type': 'others', 'count': 1, 'items': [{'id...",1 like,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Go to Vana Cuisine and order number 10 Lemon G...,0,user,NaN,Ade,male,331583,Win,NaN,https://fastly.4sqi.net/img/user/,/L32ITWGHTDBJ2PVK.jpg,NaN
0,0,NaN,https://foursquare.com/item/53e7e89411d22533ef...,1407707284,0,53e7e89411d22533ef8a64c9,en,NaN,NaN,0,[],NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Come with friends. Divide and conquer. Have ea...,0,user,NaN,Trevor,male,510020,Luu,NaN,https://fastly.4sqi.net/img/user/,/CVFWFP5FAMFLDZNU.jpg,NaN
0,4,liked,https://foursquare.com/item/5a0e576dee71206eff...,1510889325,0,5a0e576dee71206eff59c176,en,1.564879e+09,Upvoted 3 weeks ago,0,[],NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Everything is fresh, I can always count on Who...",0,user,NaN,Serafettin,male,8705833,B.,NaN,https://fastly.4sqi.net/img/user/,/C5OBCSKQUT2WVVSV.jpg,NaN
0,0,NaN,https://foursquare.com/item/525da93d11d2c6cc6b...,1381869885,0,525da93d11d2c6cc6b96ac9a,en,NaN,NaN,0,[],NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jerk Pork is delicious...,0,user,NaN,Ikik,male,38208741,JP,NaN,https://fastly.4sqi.net/img/user/,/HAM4TBL1R3WRTWHR.jpg,NaN
0,0,NaN,https://foursquare.com/item/5397e019498eaeeb1a...,1402462233,0,5397e019498eaeeb1a0a1ecb,en,NaN,NaN,0,[],NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Staff is friendly and helpful. But what surpri...,0,user,NaN,Katia,female,25011596,Villa,NaN,https://fastly.4sqi.net/img/user/,/0GLI3BCDRO0GDSF5.jpg,NaN
0,1,NaN,https://foursquare.com/item/4ec1e396003940d47f...,1321329558,0,4ec1e396003940d47fa63902,en,NaN,NaN,0,[],NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,If you're an early riser and want to avoid cro...,0,user,NaN,Richard,male,10155017,Mayede,NaN,https://fastly.4sqi.net/img/user/,/HFHMXXODU35KW4YH.jpg,NaN
0,0,liked,https://foursquare.com/item/55613c70498e653c5c...,1432435824,0,55613c70498e653c5cbd9e2c,fr,NaN,NaN,0,[],NaN,True,1.432436e+09,1439.0,55613c72498eaecfcbde3cd8,https://fastly.4sqi.net/img/general/,Foursquare for iOS,https://foursquare.com/download/#/iphone,/1763659_eNKzGKEMoyyRhLSjXkfeToYv0trma5zigG5Ca...,public,1920.0,https://fastly.4sqi.net/img/general/original/1...,Excellent food. Excellent cocktails. Excellent...,0,user,NaN,Gillian,female,1763659,Vrooman,NaN,https://fastly.4sqi.net/img/user/,/1763659-KVAFK0JT21KCLQ0I.jpg,NaN
0,0,liked,https://foursquare.com/item/5c54d7aa418686002c...,1549064106,0,5c54d7aa418686002c972243,en,NaN,NaN,0,[],NaN,True,1.549064e+09,1960.0,5c54d7adb23dfa00327c195c,https://fastly.4sqi.net/img/general/,Foursquare for Android,https://foursquare.com/download/#/android,/28585983_Ud5yFmaGGSU0XsIHBG4oGlosKHxgdeZn3iuO...,public,4032.0,https://fastly.4sqi.net/img/general/original/2...,Deja Brew had the best Tomato Basil soup in Va...,0,user,NaN,Wanda,female,28585983,Halpert,NaN,https://fastly.4sqi.net/img/user/,/RN34Y5NH130C5UON.jpg,NaN
0,0,NaN,https://foursquare.com/item/52e02cdf498ede63f2...,1390423263,0,52e02cdf498ede63f2767206,en,NaN,NaN,0,[],NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chili cheese dog is a solid pick. Meat filled ...,0,user,NaN,Jared,male,9384833,T,NaN,https://fastly.4sqi.net/img/user/,/XPUVJBQE1KDFZAL1.jpg,NaN
0,1,NaN,https://foursquare.com/item/56f8d699498ed539fc...,1459148441,0,56f8d699498ed539fc1336be,en,NaN,NaN,1,"[{'type': 'others', 'count': 1, 'items': [{'id...",1 like,True,1.459148e+09,355.0,56f8d69d498e058835983600,https://fastly.4sqi.net/img/general/,

In [34]:
# keep only columns that include likes, and anything that is associated with tips
filtered_columns = ['text', 'agreeCount'] + [col for col in dfr.columns if col.startswith('likes.')] + ['id']
df_ratings = dfr.loc[:, filtered_columns]

# clean column names by keeping only last term
df_ratings.columns = [column.split('.')[-1] for column in df_ratings.columns]

df_ratings.head()

,text,agreeCount,count,groups,summary,id
0,Go to Vana Cuisine and order number 10 Lemon G...,2,1,"[{'type': 'others', 'count': 1, 'items': [{'id...",1 like,4f85de4ee4b0a3bc1668c94e
0,Come with friends. Divide and conquer. Have ea...,0,0,[],NaN,53e7e89411d22533ef8a64c9
0,"Everything is fresh, I can always count on Who...",4,0,[],NaN,5a0e576dee71206eff59c176
0,Jerk Pork is delicious...,0,0,[],NaN,525da93d11d2c6cc6b96ac9a
0,Staff is friendly and helpful. But what surpri...,0,0,[],NaN,5397e019498eaeeb1a0a1ecb


In [35]:
df_ratings = df_ratings.drop(['groups'], axis=1)
df_ratings.head()

,text,agreeCount,count,summary,id
0,Go to Vana Cuisine and order number 10 Lemon G...,2,1,1 like,4f85de4ee4b0a3bc1668c94e
0,Come with friends. Divide and conquer. Have ea...,0,0,NaN,53e7e89411d22533ef8a64c9
0,"Everything is fresh, I can always count on Who...",4,0,NaN,5a0e576dee71206eff59c176
0,Jerk Pork is delicious...,0,0,NaN,525da93d11d2c6cc6b96ac9a
0,Staff is friendly and helpful. But what surpri...,0,0,NaN,5397e019498eaeeb1a0a1ecb


In [36]:
# map with clusters and tips

# create map
map_clusters = folium.Map(location=[latitude3, longitude3], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, tip, like in zip(df_vanc['lat'], df_vanc['lng'], df_new['name'], df_vanc['Cluster Labels'], df_ratings['text'], df_ratings['summary']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + '   ' + str(like) + '    ' + str(tip), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters